# 协同过滤算法
## 1. 基于用户的协同过滤算法 UserCF



In [5]:
'''
Descripttion: my code for learning
Author: chenggong Pan
Date: 2024-03-01 15:02:45
LastEditors: chenggong Pan
LastEditTime: 2024-03-01 15:25:05
'''
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def loadData():
    """
    加载用户数据。
    
    该函数没有参数。
    
    返回:
    dict: 包含用户及其对应等级评分的字典。每个用户是一个键，其值是另一个字典，该字典以课程名称为键，以评分等级为值。
    """
    users = {'Alice': {'A': 5, 'B': 3, 'C': 4, 'D': 4},
             'user1': {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
             'user2': {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
             'user3': {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
             'user4': {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
             }
    return users

# 加载用户数据
user_data = loadData()

# 初始化用户相似度矩阵，使用pandas的DataFrame，初始值为单位矩阵
similarity_matrix = pd.DataFrame(
    np.identity(len(user_data)),  # 生成单位矩阵，其大小根据用户数量确定
    index=user_data.keys(),  # 设置行索引为用户键
    columns=user_data.keys()  # 设置列索引为用户键
)

# 遍历每条用户-物品评分数据
for u1, items1 in user_data.items():
    for u2, items2 in user_data.items():
        if u1 == u2:
            continue
        vec1, vec2 = [], []
        for item, rating1 in items1.items():
            rating2 = items2.get(item, -1)
            if rating2 == -1:
                continue
            vec1.append(rating1)
            vec2.append(rating2)
        # 计算不同用户之间的皮尔逊相关系数
        similarity_matrix[u1][u2] = np.corrcoef(vec1, vec2)[0][1]
        # 计算不同用户的余弦相似度
        # similarity_matrix[u1][u2] = cosine_similarity(vec1, vec2)[0][1]

print(similarity_matrix)



target_user = 'Alice'
num = 2
# 由于最相似的用户为自己，去除本身
sim_users = similarity_matrix[target_user].sort_values(ascending=False)[1:num+1].index.tolist()
print(f'与用户{target_user}最相似的{num}个用户为：{sim_users}')


          Alice     user1     user2     user3     user4
Alice  1.000000  0.852803  0.707107  0.000000 -0.792118
user1  0.852803  1.000000  0.467707  0.489956 -0.900149
user2  0.707107  0.467707  1.000000 -0.161165 -0.466569
user3  0.000000  0.489956 -0.161165  1.000000 -0.641503
user4 -0.792118 -0.900149 -0.466569 -0.641503  1.000000
与用户Alice最相似的2个用户为：['user1', 'user2']


In [6]:
# 预测用户 Alice 对物品 E 的评分
weighted_scores = 0.
corr_values_sum = 0.

target_item = 'E'
# 基于皮尔逊相关系数预测用户评分
# 遍历与目标用户相似的用户，计算加权评分
for user in sim_users:
    corr_value = similarity_matrix[target_user][user]
    user_mean_rating = np.mean(list(user_data[user].values()))
    # 计算加权评分
    weighted_scores += corr_value * (user_data[user][target_item] - user_mean_rating)
    corr_values_sum += corr_value

# 计算目标用户对物品 E 的预测评分
# 对物品 E 的预测评分 = 目标用户均值 + 相关评分的加权和 / 相关评分的和
target_user_mean_rating = np.mean(list(user_data[target_user].values()))
target_item_pred = target_user_mean_rating + weighted_scores / corr_values_sum
print(f'用户{target_user}对物品{target_item}的预测评分为：{target_item_pred}')


用户Alice对物品E的预测评分为：4.871979899370592


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# 导入cosine_similarity函数，用于计算两个向量之间的余弦相似度

i = [1, 0, 0, 0]  # 定义第一个向量i
j = [1, 0, 1, 0]  # 定义第二个向量j

# 计算向量i和向量j之间的余弦相似度
cosine_similarity([i, j])


array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

In [1]:
# 皮尔逊相似度
from scipy.stats import pearsonr

i = [1, 0, 0, 0]
j = [1, 0.5, 0.5, 0]
pearsonr(i, j)


PearsonRResult(statistic=0.8164965809277258, pvalue=0.1835034190722742)

In [3]:
# 余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 定义向量 A 和 B
A = np.array([[0, 1, 0, 1]])  # 向量 A = [b, d]
B = np.array([[1, 1, 1, 0]])  # 向量 B = [a, b, c]

# 计算余弦相似度
cosine_sim = cosine_similarity(A, B)

print("余弦相似度为:", cosine_sim[0][0])


# from math import sqrt

# # 定义向量 A 和 B
# A = {'b': 1, 'd': 1}
# B = {'a': 1, 'b': 1, 'c': 1}

# # 计算向量 A 和 B 的点积
# dot_product = sum(A.get(key, 0) * B.get(key, 0) for key in set(A) | set(B))

# # 计算向量 A 和 B 的模长
# magnitude_A = sqrt(sum(value**2 for value in A.values()))
# magnitude_B = sqrt(sum(value**2 for value in B.values()))

# # 计算余弦相似度
# cosine_similarity = dot_product / (magnitude_A * magnitude_B)

# print("余弦相似度为:", cosine_similarity)



余弦相似度为: 0.408248290463863


In [6]:
'''
Author: cgp 2986779260@qq.com
Date: 2024-03-01 15:02:45
LastEditors: cgp 2986779260@qq.com
LastEditTime: 2024-03-01 21:45:58
FilePath: \jupyter\AI\RS\algorithm\cf.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
print("hello")
# print("傻逼")




hello


## 2. 基于物品的协同过滤算法 ItmeCF

